In [1]:
import sys
sys.path.append("/home/kelab/m6AAIpy2")

In [2]:
from keras.models import load_model
from pkg_resources import resource_filename
import numpy as np
import pandas as pd
from Bio.Seq import Seq
import keras.backend as kb

Using TensorFlow backend.


In [3]:
def one_hot_encode(seq):

    map = np.asarray([[0, 0, 0, 0],
                      [1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])

    seq = seq.upper().replace('A', '\x01').replace('C', '\x02')
    seq = seq.replace('G', '\x03').replace('T', '\x04').replace('N', '\x00')

    return map[np.fromstring(seq, np.int8) % 5]

In [4]:
def categorical_crossentropy_2d(y_true, y_pred):
    # Standard categorical cross entropy for sequence outputs

    return - kb.mean(y_true[:, :, 0]*kb.log(y_pred[:, :, 0]+1e-10)
                   + y_true[:, :, 1]*kb.log(y_pred[:, :, 1]+1e-10))

In [5]:
context = 10000

In [6]:
paths = ('/home/kelab/iM6A/mouseRAC10000_c{}.h5'.format(x) for x in range(1, 6))

In [7]:
models = [load_model(y, custom_objects={'categorical_crossentropy_2d': categorical_crossentropy_2d}) for y in paths]

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
models

### Read data

In [1]:
import numpy as np
import pandas as pd

In [10]:
Fasta = pd.read_csv("Temp/mm10_LastIntron_Fasta.csv")

In [ ]:
Fasta = Fasta[Fasta["exonCount"]>=3]
Fasta = Fasta.reset_index(drop = True)

### Select positive strand

In [12]:
Fasta_Pos = Fasta[Fasta["strand"]=="+"]

In [13]:
Fasta_Pos = Fasta_Pos.reset_index(drop = True)

In [14]:
Fasta_Pos

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,...,Group,Sequence,GeneLength,LastExonLength,LastIntronLength,SecondToLastLength,Pre,PreSequence,LastIntronSequence,LastExonSequence
0,SGIP1,chr1,+,66999964,67213982,67000041,67208778,25,"66999964,67091529,67098752,67101626,67105459,6...","67000051,67091593,67098777,67101698,67105516,6...",...,2,CTTAGCTGGGACCTGGAATCGTATCCTCCTGTGTTTTTTCAGACTC...,214018,5227,1636,165,207155,CTTAGCTGGGACCTGGAATCGTATCCTCCTGTGTTTTTTCAGACTC...,GTAAATGAGTATCTTGATTTTTTCATTTGCGAAAATAATGTTGCCT...,GAAAATACTTGGCAGATAACTAATGAAATCTTATGCAAGGATTTGG...
1,SLC45A1,chr1,+,8384389,8404227,8384389,8404073,8,"8384389,8385357,8385877,8390268,8395496,839787...","8384786,8385450,8386102,8390996,8395650,839805...",...,2,ATGATCCCCGCAGCCAGCAGCACCCCGCCGGGAGATGCCCTCTTCC...,19838,421,4048,206,15369,ATGATCCCCGCAGCCAGCAGCACCCCGCCGGGAGATGCCCTCTTCC...,GTAAGTGCTCTCCCTTGTCTTGCTTCGGGTCTGCTTCTTGGAGAAA...,TTTGCAGGGTCCAGTGCGGACGGCACCCGGCGGGGCATGGGCGTGG...
2,NECAP2,chr1,+,16767166,16786573,16767256,16785385,8,"16767166,16770126,16774364,16774554,16775587,1...","16767348,16770227,16774469,16774636,16775696,1...",...,2,GAAACCTGGTCAGAGAGTCGCACCGCTTCCGTCCGTCGGACAGAGG...,19407,1237,2948,76,15222,GAAACCTGGTCAGAGAGTCGCACCGCTTCCGTCCGTCGGACAGAGG...,GTAAGGAGGGCCATGTTCTGCGGAGGGGCTGGGGCCAGGGCCGTTG...,ATCAACTTCCAGCCAGACCCAGCCAGGCACAGGCTGGGTCCAGTTC...
3,CLIC4,chr1,+,25071847,25170815,25072044,25167428,6,"25071847,25124232,25140584,25153500,25166350,2...","25072116,25124342,25140710,25153607,25166532,2...",...,2,AAGCGGCTCGGGCTGCGGCTGGCTCAGAGTGGCGCGGGGGGCGTGG...,98968,3552,731,182,94685,AAGCGGCTCGGGCTGCGGCTGGCTCAGAGTGGCGCGGGGGGCGTGG...,GTAGGTCTGTAGGGGTTGATGTCGCATTGCCATTACCTTGTATTGT...,GTGGTGGCCAAAAAATATCGCAACTTTGATATTCCAAAAGAAATGA...
4,ADC,chr1,+,33546713,33585995,33547850,33585783,12,"33546713,33546988,33547201,33547778,33549554,3...","33546895,33547109,33547413,33547955,33549728,3...",...,2,TGGGCGGGGTTATATAAGCCAGCGGCGGGCGGAAGGCGGGGCGTGG...,39282,351,1927,215,37004,TGGGCGGGGTTATATAAGCCAGCGGCGGGCGGAAGGCGGGGCGTGG...,GTAAGAGGGCCCTGCTGGAAAATGGGGGTATGGGGAGGAACTGGGC...,GGAAGCGCTGCGAAGGCAGCTGATGGCTGCAGAACAGGAGGATGAC...
5,PRUNE,chr1,+,150980952,151008189,150981108,151006710,8,"150980952,150990287,150990942,150997086,150997...","150981147,150990380,150991145,150997271,150998...",...,2,GGAGGCCGATTCGCCGTGTGGCGGGTTCGAGTCCCGCCTCCTGACT...,27237,1908,4861,159,20468,GGAGGCCGATTCGCCGTGTGGCGGGTTCGAGTCCCGCCTCCTGACT...,GTGAGTCTGTGTCCCTTCTCCAACCTAAGAGCCTTACAGAGTCTGC...,ATCTGTGAAGTCCTGGAACGCTCCCACTCTCCACCCCTGAAGCTGA...
6,C1orf21,chr1,+,184356191,184598154,184446643,184588690,6,"184356191,184446519,184476721,184559872,184567...","184356502,184446737,184476816,184559949,184567...",...,2,CCCACTCCCGGGGGGACGTTCCGTGCCGCGGCCGCCGCGGCCGCTG...,241963,9503,21056,61,211404,CCCACTCCCGGGGGGACGTTCCGTGCCGCGGCCGCCGCGGCCGCTG...,GTAAGAAGTGAAATAAATAACCTACATGTTTCAAGGGAAGTATAGT...,GGTCGGGATTACTGTTCGGAAGAAGAGGATATCACATAGCACCAAT...
7,PRKCZ,chr1,+,1981908,2116834,1982069,2116448,18,"1981908,1986879,1987922,1990979,2066700,207564...","1982140,1987001,1988012,1991030,2066786,207578...",...,2,gccccgcgcgccgccggAGTTCCGCGGAGTTGACCGGGTCGGCGCC...,134926,474,223,116,134229,gccccgcgcgccgccggAGTTCCGCGGAGTTGACCGGGTCGGCGCC...,GTGAGTCCCACTGGGTGCGGGTCCCTGGAGCACCCCTCGGGCAGCC...,GGATGCCATAAAGAGGATCGACCAGTCAGAGTTCGAAGGCTTTGAG...
8,PRDM16,chr1,+,2985741,3355183,2985823,3350375,17,"2985741,3102688,3160650,3301715,3313054,331935...","2985860,3103038,3160701,3301850,3313157,331956...",...,2,CTGACAATGCTGGGGAGATGAAGATAGTGTGTGGCTGCTTCTGGAC...,369442,4886,1593,175,362963,CTGACAATGCTGGGGAGATGAAGATAGTGTGTGGCTGCTTCTGGAC...,GTAGGTACCCGCCAGAGCCCCTCCCCCACCCCACCTGGCCTCCTCA...,GGTTCTCTGGACGCTTGGTTGAAGGTCACTGGAGCCACGTCGGAGT...
9,CAMTA1,chr1,+,6845383,7829766,6845590,7826551,23,"6845383,6880240,6885151,7151363,7309550,752788...","6845635,6880310,6885270,7151431,7309686,752796...",...,2,GACGCTCCTCCCGGAGAGTAGTGAGACCCCTGGTGCGGGGCGATTG...,984383,3248,10790,31,970345,GACGCTCCTCCCGGAGAGTAGTGAGACCCCTGGTGCGGGGCG

In [15]:
Sequence = Fasta_Pos["Sequence"].tolist()

In [2]:
df = pd.DataFrame(range(-1000,1001), columns=["index"])

In [ ]:
for i in range(len(Fasta_Pos)):
    
    # Define exon location   
    Exon = []
    a = Fasta_Pos.loc[i, "exonStarts"].split(",")[0:-1]
    b = Fasta_Pos.loc[i, "exonEnds"].split(",")[0:-1]
    A = [int(u) for u in a]
    B = [int(v) for v in b]
    Exon = A + B
    Exon = sorted(Exon)
    
    print(i)
    
    # Define length of exon and intron
    Length = []
    for j in range(1,len(Exon),1):
        length = Exon[j] - Exon[-1 + j]
        Length.append(length)
    
    CumSum = []
    Sum = 0
    for k in Length:
        Sum = Sum + k
        CumSum.append(Sum)    
    
    
    # Define sequence of exon and intron
    input_sequence = Sequence[i]
    
    First = [input_sequence[0:CumSum[0]]]
    for m in range(1,len(CumSum),1):
        seq = input_sequence[(CumSum[m-1]):(CumSum[m])]
        First.append(seq)
    
    # Define sequence of exon
    sequence = ""
    Sum = 0
    CumSum = []
    for n in range(0,len(First),2):
        seq = First[n]
        Sum = Sum + len(seq)
        sequence = sequence + seq
        CumSum.append(Sum)  
    
    # Prediction
    x = one_hot_encode('N'*(context//2) + sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    m6AAI_prob = y[0, :, 1]
    m6AAI_prob = m6AAI_prob.tolist()

        
    iM6A_prob = m6AAI_prob 
    Start = len(iM6A_prob)-Fasta_Pos.loc[i, "LastExonLength"]    ############ Start position
    
    Pre = iM6A_prob[0:Start]
    Last = iM6A_prob[Start:]
    
    if len(Pre) < 1000:
        Pre = [0]*(1000-Start) + Pre
    if len(Pre) >= 1000:
        Pre = Pre[-1000:]
    
    if len(Last) < 1001:
        Last = Last + [0]*(1001-len(Last))
    if len(Last) >= 1001:
        Last = Last[0:1001]
    
    New = np.array(Pre + Last)
    New = pd.DataFrame({Fasta_Pos.loc[i,"name"]:New})
    
    df = pd.merge(df, New, left_index=True, right_index=True)        

### Select negative strand

In [ ]:
Fasta_Neg = Fasta[Fasta["strand"]=="-"]

In [ ]:
Fasta_Neg = Fasta_Neg.reset_index(drop = True)

In [ ]:
Fasta_Neg

In [ ]:
Sequence = Fasta_Neg["Sequence"].tolist()

In [ ]:
for i in range(len(Fasta_Neg)):
    
    # Define exon location
    Exon = []
    a = Fasta_Neg.loc[i, "exonStarts"].split(",")[0:-1]
    b = Fasta_Neg.loc[i, "exonEnds"].split(",")[0:-1]
    A = [int(u) for u in a]
    B = [int(v) for v in b]
    Exon = A + B
    Exon = sorted(Exon)
    
    print(i)
    
    # Define length of exon and intron
    Length = []
    for j in range(1,len(Exon),1):
        length = Exon[j] - Exon[-1 + j]
        Length.append(length)
    Length = Length[::-1]
    
    CumSum = []
    Sum = 0
    for k in Length:
        Sum = Sum + k
        CumSum.append(Sum)
   
    # Define sequence of exon and intron
    input_sequence = Sequence[i]
    
    First = [input_sequence[0:CumSum[0]]]
    for m in range(1,len(CumSum),1):
        seq = input_sequence[(CumSum[m-1]):(CumSum[m])]
        First.append(seq)
        
    # Define sequence of exon
    sequence = ""
    Sum = 0
    CumSum = []
    for n in range(0,len(First),2):
        seq = First[n]
        Sum = Sum + len(seq)
        sequence = sequence + seq
        CumSum.append(Sum)          
            
    # Prediction
    x = one_hot_encode('N'*(context//2) + sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    m6AAI_prob = y[0, :, 1]
    m6AAI_prob = m6AAI_prob.tolist()        

    iM6A_prob = m6AAI_prob
    Start = len(iM6A_prob) - Fasta_Neg.loc[i, "LastExonLength"]    ############ Start position
    
    Pre = iM6A_prob[0:Start]
    Last = iM6A_prob[Start:]
    
    if len(Pre) < 1000:
        Pre = [0]*(1000-Start) + Pre
    if len(Pre) >= 1000:
        Pre = Pre[-1000:]
    
    if len(Last) < 1001:
        Last = Last + [0]*(1001-len(Last))
    if len(Last) >= 1001:
        Last = Last[0:1001]
    
    New = np.array(Pre + Last)
    New = pd.DataFrame({Fasta_Neg.loc[i,"name"]:New})
    
    df = pd.merge(df, New, left_index=True, right_index=True)        

In [ ]:
df

In [ ]:
df.drop(["index"], axis=1, inplace=True)
df = df.T

In [ ]:
df.loc["Sum"] = df.sum()
df = df.T

In [ ]:
Value = df[["Sum"]]

In [ ]:
Value.head(5)

In [ ]:
df.drop(["Sum"], axis=1, inplace=True)
df = df.T

In [ ]:
df.loc["Number"] = (df > 0).sum()
df = df.T

In [ ]:
Number = df[["Number"]]

In [ ]:
Number.head(5)

In [ ]:
Data = pd.concat([Value, Number], axis=1)

In [ ]:
Data["Mean"] = Data["Sum"]/Data["Number"]

In [ ]:
Data

In [ ]:
Data = Data.head(2000)
Sum = []
Number = []

for j in range(0,len(Data),10):

    a = Data.loc[j:(j+9),"Sum"]
    b = np.mean(a, axis=0)
    Sum.append(b)
    
    a = Data.loc[j:(j+9),"Number"]
    b = np.mean(a, axis=0)
    Number.append(b)

Result = pd.DataFrame({"Index":range(-1000,1000,10), "Sum":Sum, "Number":Number})
Result["Mean"] = Result["Sum"]/Result["Number"] 

In [ ]:
Result

In [ ]:
Result.to_csv("Mouse_iM6A_LastExonStart_10interval_Intronless.csv", index=0)